<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align="center"><font size="5">Classification with Python</font></h1>


In this notebook we try to practice all the classification algorithms that we have learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Let's first load required libraries:


In [ ]:
!pip install scikit-learn==0.23.1
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset


This dataset is about past loans. The **Loan_train.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
| -------------- | ------------------------------------------------------------------------------------- |
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |


Let's download the dataset


In [ ]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

### Load Data From CSV File


In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object


In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing


Let’s see how many of each class is in our data set


In [ ]:
df['loan_status'].value_counts()
df.Principal, df.loan_status

260 people have paid off the loan on time while 86 have gone into collection


Let's plot some columns to underestand data better:


In [ ]:
# notice: installing seaborn might takes a few minutes
#!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 25)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

bins = np.linspace(df.terms.min(), df.terms.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'terms', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Let's look at the day of the week people get the loan


In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 8)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week don't pay it off, so let's use Feature binarization to set a threshold value less than day 4


In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()
# Terms greater than 10 don't pay off
df['termbin'] = df['terms'].apply(lambda x: 1 if (x>10)  else 0)
df.head()

# Principal greater than 400 don't pay off
df['Prinbin'] = df['Principal'].apply(lambda x: 1 if (x>400)  else 0)
df.head()

## Convert Categorical features to numerical values


Let's look at gender:


In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Let's convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding

#### How about education?


In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Features before One Hot Encoding


In [ ]:
df[['Principal','terms','age','Gender','education']].head()


#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
# Drop master since it is a 50/50 split. Poor indicator
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()
X = Feature
X[0:5]

### Feature Selection


Let's define feature sets, X:


In [ ]:
X = Feature
X[0:5]

What are our lables?


In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data


Data Standardization give data zero mean and unit variance (technically should be done after train test split)


In [ ]:
# Data stardardization also converts the Feature set into a numeric array
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification


Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:

*   K Nearest Neighbor(KNN)
*   Decision Tree
*   Support Vector Machine
*   Logistic Regression

\__ Notice:\__

*   You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
*   You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
*   You should include the code of the algorithm in the following cells.


# K Nearest Neighbor(KNN)

Notice: You should find the best k to build the model with the best accuracy.\
**warning:** You should not use the **loan_test.csv** for finding the best k, however, you can split your train_loan.csv into train and test to find the best **k**.


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
# Drop master since it is a 50/50 split. Poor indicator
Feature.drop(['Master or Above'], axis = 1,inplace=True)
X = Feature
y = df['loan_status'].values
y[0:5]
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]
Xknn = X
yknn = y
from sklearn.model_selection import train_test_split
Xknn_train, Xknn_test, yknn_train, yknn_test = train_test_split( Xknn, yknn, test_size=0.2, random_state=4)
print ('Train set:', Xknn_train.shape,  yknn_train.shape)
print ('Test set:', Xknn_test.shape,  yknn_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(Xknn_train,yknn_train)
    yhat=neigh.predict(Xknn_test)
    mean_acc[n-1] = metrics.accuracy_score(yknn_test, yhat)

    
    std_acc[n-1]=np.std(yhat==yknn_test)/np.sqrt(yhat.shape[0])

mean_acc
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.fill_between(range(1,Ks),mean_acc - 3 * std_acc,mean_acc + 3 * std_acc, alpha=0.10,color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1)
neigh = KNeighborsClassifier(n_neighbors = 7).fit(Xknn_train,yknn_train)
yhatknn=neigh.predict(Xknn_test)
print("Accuracy = ", metrics.accuracy_score(yknn_test, yhatknn))
yknn_test, yhatknn

# Decision Tree


In [ ]:
#Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = df[['Prinbin','termbin','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
# Drop master since it is a 50/50 split. Poor indicator
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Xdt = Feature
Xdt= preprocessing.StandardScaler().fit(Xdt).transform(Xdt)
ydt = df['loan_status'].values
Xdt_train, Xdt_test, ydt_train, ydt_test = train_test_split( Xdt, ydt, test_size=0.1, random_state=4)
print ('Train set:', Xdt_train.shape,  ydt_train.shape)
print ('Test set:', Xdt_test.shape,  ydt_test.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree
# Only parameter that can be varied is max_depth
loanTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
loanTree.fit(Xdt_train,ydt_train)
predTree = loanTree.predict(Xdt_test)
print (predTree )
print (ydt_test )

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(ydt_test, predTree))
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y
tree.plot_tree(loanTree)
plt.show()

# Support Vector Machine


In [ ]:
Feature = df[['Principal','terms','age','Gender','dayofweek']]
Feature = df[['Prinbin','termbin','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
# Drop master since it is a 50/50 split. Poor indicator
Feature.drop(['Master or Above'], axis = 1,inplace=True)
X = Feature
print(X.dtypes)
#X= preprocessing.StandardScaler().fit(X).transform(X)
#X[0:5]
Xsvm = X

# For SVM, the labels should be integer type
y = df['loan_status'].values
le_loan = preprocessing.LabelEncoder()
le_loan.fit(['PAIDOFF','COLLECTION'])
ysvm = le_loan.transform(y)

In [ ]:
# Split data into train and test.
Xsvm_train, Xsvm_test, ysvm_train, ysvm_test = train_test_split( Xsvm, ysvm, test_size=0.1, random_state=4)
print ('Train set:', Xsvm_train.shape,  ysvm_train.shape)
print ('Test set:', Xsvm_test.shape,  ysvm_test.shape)

In [ ]:
from sklearn import svm
#clf = svm.SVC(kernel='linear')
#clf = svm.SVC(kernel='poly')
clf = svm.SVC(kernel='rbf')
#clf = svm.SVC(kernel='sigmoid')
clf.fit(Xsvm_train, ysvm_train) 
yhatsvm = clf.predict(Xsvm_test)
print("SVM's Accuracy: ", metrics.accuracy_score(ysvm_test, yhatsvm))
yhatsvm,ysvm_test

# Logistic Regression


In [ ]:
#Feature = df[['Principal','terms','age','Gender','dayofweek']]
Feature = df[['Prinbin','termbin','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
X = Feature
X= preprocessing.StandardScaler().fit(X).transform(X)
Xlr = X

# For LR, the labels should be integer type
y = df['loan_status'].values
le_loan = preprocessing.LabelEncoder()
le_loan.fit(['PAIDOFF','COLLECTION'])
ylr = le_loan.transform(y)

In [ ]:
# Split data into train and test.
Xlr_train, Xlr_test, ylr_train, ylr_test = train_test_split( Xlr, ylr, test_size=0.1, random_state=4)
print ('Train set:', Xlr_train.shape,  ylr_train.shape)
print ('Test set:', Xlr_test.shape,  ylr_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
#LR = LogisticRegression(C=0.01, solver='lbfgs').fit(Xlr_train,ylr_train)
LR = LogisticRegression(C=0.01, solver='newton-cg').fit(Xlr_train,ylr_train)
#LR = LogisticRegression(C=0.01, solver='liblinear').fit(Xlr_train,ylr_train)
#LR = LogisticRegression(C=0.01, solver='sag').fit(Xlr_train,ylr_train)
#LR = LogisticRegression(C=0.01, solver='saga').fit(Xlr_train,ylr_train)
yhatlr = LR.predict(Xlr_test)
yhatlr_prob = LR.predict_proba(Xlr_test)
print("LR's Accuracy: ", metrics.accuracy_score(ylr_test, yhatlr))
#from sklearn.metrics import jaccard_score
#jaccard_score(ylr_test, yhat,pos_label=0)
#from sklearn.metrics import classification_report, confusion_matrix
#print (classification_report(y_test, yhat))
from sklearn.metrics import log_loss
print(log_loss(ylr_test, yhatlr_prob))
yhatlr,ylr_test

# Model Evaluation using Test set


In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss


First, download and load the test set:


In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation


In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
#preprocessing
# replicate the preprocessing steps from df
#test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['termbin'] = test_df['terms'].apply(lambda x: 1 if (x>10)  else 0)
test_df['Prinbin'] = test_df['Principal'].apply(lambda x: 1 if (x>400)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.dtypes

#Preprocessing for KNN
Feature = test_df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()
Xknn = Feature
Xlr= preprocessing.StandardScaler().fit(Xknn).transform(Xknn)
yknn = test_df['loan_status'].values
print(Xknn.shape,yknn.shape)

#Preprocessing for Decision Tree
Feature = test_df[['Prinbin','termbin','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Xdt = Feature
Xdt= preprocessing.StandardScaler().fit(Xdt).transform(Xdt)
ydt = test_df['loan_status'].values
print(Xdt.shape,ydt.shape)

#Preprocessing for SVM
Feature = test_df[['Principal','terms','age','Gender','dayofweek']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Xsvm = Feature
ysvm = test_df['loan_status'].values
le_loan = preprocessing.LabelEncoder()
le_loan.fit(['PAIDOFF','COLLECTION'])
ysvm = le_loan.transform(ysvm)
print(Xsvm.shape,ysvm.shape)

#Preprocessing for LR
Feature = test_df[['Prinbin','termbin','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Xlr = Feature
X= preprocessing.StandardScaler().fit(Xlr).transform(Xlr)
ylr = test_df['loan_status'].values
le_loan = preprocessing.LabelEncoder()
le_loan.fit(['PAIDOFF','COLLECTION'])
ylr = le_loan.transform(ylr)
print(Xlr.shape,ylr.shape)

In [ ]:
#Applying fitted model to data
# KNN
yhatknn=neigh.predict(Xknn)
yknn,yhatknn
print("KNN's Accuracy = ", metrics.accuracy_score(yknn, yhatknn))
# Decision tree
predTree = loanTree.predict(Xdt)
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(ydt, predTree))
# SVM
yhatsvm = clf.predict(Xsvm)
print("SVM's Accuracy: ", metrics.accuracy_score(ysvm, yhatsvm))
# LR
yhatlr = LR.predict(Xlr)
yhatlr_prob = LR.predict_proba(Xlr)
print("LR's Accuracy: ", metrics.accuracy_score(ylr, yhatlr))
#ysvm_test,yhatknn,predTree,yhatsvm,yhatlr

In [ ]:
print('         Algorithm      Jaccard             F1-Score            Logloss')
print('               KNN  ',jaccard_score(yknn, yhatknn,pos_label='PAIDOFF'),f1_score(yknn, yhatknn, average='weighted'),'         NA' )
print('     Decision Tree  ',jaccard_score(ydt, predTree,pos_label='PAIDOFF'),f1_score(ydt, predTree, average='weighted'),'         NA' )
print('               SVM  ',jaccard_score(ysvm, yhatsvm,pos_label=1),f1_score(ysvm, yhatsvm, average='weighted'),'         NA' )
print('LogisticRegression  ',jaccard_score(ylr, yhatlr,pos_label=1),f1_score(ylr, yhatlr, average='weighted'),log_loss(ylr, yhatlr_prob) )

# Report

You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Jaccard | F1-score | LogLoss |
| ------------------ | ------- | -------- | ------- |
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description                                                             |
| ----------------- | ------- | ------------- | ------------------------------------------------------------------------------ |
| 2020-10-27        | 2.1     | Lakshmi Holla | Made changes in import statement due to updates in version of  sklearn library |
| 2020-08-27        | 2.0     | Malika Singla | Added lab to GitLab                                                            |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
